In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump

In [15]:
# CSVファイルを読み込む
df = pd.read_csv('scraped_data.csv', index_col=0)

#df.head(2)

In [3]:
import configparser
from sqlalchemy import create_engine

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['database']['user']
db_password = config['database']['password']
db_host = config['database']['host']
db_name = config['database']['name']
db_port = config['database']['port']

# PostgreSQLデータベースへの接続情報
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
df = pd.read_sql_table('rental_properties', engine)

In [18]:
drop_cols = ['posting_date'] # , 'removed_date', 'predict_cost'
df.drop(drop_cols, axis=1, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    2073 non-null   object 
 1   building_name    2073 non-null   object 
 2   address          2073 non-null   object 
 3   access_1         2073 non-null   object 
 4   access_2         2062 non-null   object 
 5   access_3         2023 non-null   object 
 6   age              2073 non-null   object 
 7   building_floors  2073 non-null   object 
 8   room_floor       2073 non-null   object 
 9   rent             2073 non-null   float64
 10  management_fee   2073 non-null   float64
 11  deposit          2073 non-null   float64
 12  gratuity         2073 non-null   float64
 13  layout           2073 non-null   object 
 14  area             2073 non-null   float64
 15  url              2073 non-null   object 
 16  actual_cost      2073 non-null   float64
dtypes: float64(6),

# 前処理
1. 実質負担額を作成する
   1. rent +  management_fee +  (deposit / 48)
2. access_1, access_2, access_3から最寄駅ではなくて桜山駅からの時間を取得する
2. 数値型に変換する
   1. age, building_floors, room_floor
   2. area

In [20]:
# 桜山駅からの最短時間を取得する関数
def get_min_time_to_sakurayama(access_str):
    if pd.isnull(access_str):
        return np.nan
    times = [int(s.split('歩')[1].replace('分', '')) for s in access_str.split('/') if '桜山駅' in s]
    if times:
        return min(times)
    else:
        return np.nan

# access_1, access_2, access_3から桜山駅までの最短時間を計算
df['min_time_to_sakurayama'] = df[['access_1', 'access_2', 'access_3']].apply(
    lambda x: min(
        filter(pd.notnull, [get_min_time_to_sakurayama(x['access_1']), get_min_time_to_sakurayama(x['access_2']), get_min_time_to_sakurayama(x['access_3'])]),
        default=np.nan
    ),
    axis=1
)

In [21]:
# '築新'または'新'を'0'年として扱うための処理を追加
df['age'] = df['age'].str.replace('築新', '0').str.replace('新', '0').str.replace('築', '').str.replace('年', '')


# 空の値を0に置き換え
df['age'] = df['age'].replace('', '0')

# 整数型に変換
df['age'] = df['age'].astype(int)

# 建物の階数と部屋の階についても同様に処理
df['building_floors'] = df['building_floors'].str.extract('(\d+)')[0].fillna('0').astype(int)
df['room_floor'] = df['room_floor'].str.extract('(\d+)')[0].fillna('0').astype(int)

# ここでのfillna('0')は、抽出した結果がNaN（該当する数値がない場合）の場合に0を割り当てるために使用しています。

In [22]:
df.head()

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,actual_cost,min_time_to_sakurayama
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,8,2,1,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000062206827/?bc=...,58614.583333,3
1,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,8,2,1,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000088370773/?bc=...,58614.583333,3
2,賃貸マンション,ヴィルクレア桜山,愛知県名古屋市瑞穂区桜見町１,地下鉄桜通線/桜山駅 歩2分,地下鉄桜通線/瑞穂区役所駅 歩12分,地下鉄鶴舞線/御器所駅 歩17分,2,14,9,74000.0,8000.0,74000.0,74000.0,1K,29.97,https://suumo.jp/chintai/jnc_000088683433/?bc=...,85083.333333,2
3,賃貸マンション,ヴィルクレア桜山,愛知県名古屋市瑞穂区桜見町１,地下鉄桜通線/桜山駅 歩2分,地下鉄桜通線/瑞穂区役所駅 歩12分,地下鉄鶴舞線/御器所駅 歩17分,2,14,13,155000.0,8000.0,155000.0,155000.0,2LDK,60.00,https://suumo.jp/chintai/jnc_000088465047/?bc=...,169458.333333,2
4,賃貸マンション,ヴィルクレア桜山,愛知県名古屋市瑞穂区桜見町１,地下鉄桜通線/桜山駅 歩2分,地下鉄桜通線/瑞穂区役所駅 歩12分,地下鉄鶴舞線/御器所駅 歩17分,2,14,13,160000.0,8000.0,160000.0,160000.0,2LDK,60.00,https://suumo.jp/chintai/jnc_000087306700/?bc=...,174666.666667,2


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_type           2073 non-null   object 
 1   building_name           2073 non-null   object 
 2   address                 2073 non-null   object 
 3   access_1                2073 non-null   object 
 4   access_2                2062 non-null   object 
 5   access_3                2023 non-null   object 
 6   age                     2073 non-null   int64  
 7   building_floors         2073 non-null   int64  
 8   room_floor              2073 non-null   int64  
 9   rent                    2073 non-null   float64
 10  management_fee          2073 non-null   float64
 11  deposit                 2073 non-null   float64
 12  gratuity                2073 non-null   float64
 13  layout                  2073 non-null   object 
 14  area                    2073 non-null   

## object型の処理
- エンコーディングする
  - property_type
  - address
  - layout
- dropする
  - building_name
  - access_1
  - access_2
  - access_3
  - url
  - posting_date
  - predictions_rf
  - predictions_gb

In [24]:
# エンコーディングするカラム
categorical_cols = ['property_type', 'address', 'layout']

# One-hotエンコーディング
df = pd.get_dummies(df, columns=categorical_cols)

In [25]:
# 不要なカラムを削除
drop_cols = ['building_name', 'access_1', 'access_2', 'access_3', 'url']
df.drop(drop_cols, axis=1, inplace=True)

In [26]:
df.columns = df.columns.astype(str)

# モデルの訓練

In [29]:
# 特徴量とターゲット変数に分割
X = df #.drop('actual_cost', axis=1)  # actual_cost以外のすべてのカラムを特徴量として使用
y = df['actual_cost']  # actual_costをターゲット変数として使用

# データセットの分割（例：訓練データ80%、テストデータ20%）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# モデルの訓練とテスト
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

for name, model in models.items():
    # モデルの訓練
    model.fit(X_train, y_train)
    
    # テストデータで予測
    predictions = model.predict(X_test)
    
    # RMSEの計算
    rmse_score = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"{name}: Test RMSE: {rmse_score:.4f}")
    
    # モデルの保存
    dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')

Random Forest: Test RMSE: 1237.2342
Gradient Boosting: Test RMSE: 400.8682
